In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [14]:
%%javascript
$('.nbp-app-bar').toggle()

<IPython.core.display.Javascript object>

## Check out the 5 datasets

In [4]:
path = '../kaggle_data/input/'
raw = pd.read_csv(path+'spray.csv')

In [5]:
raw.head()

,Date,Time,Latitude,Longitude
0,2011-08-29,6:56:58 PM,42.391623,-88.089163
1,2011-08-29,6:57:08 PM,42.391348,-88.089163
2,2011-08-29,6:57:18 PM,42.391022,-88.089157
3,2011-08-29,6:57:28 PM,42.390637,-88.089158
4,2011-08-29,6:57:38 PM,42.390410,-88.088858


In [6]:
raw.shape

(14835, 4)

In [12]:
# How many spray dates are there?
raw['Date'].nunique()

10

In [13]:
# Spraying only occurred in 10 days over 2 years (mostly in 2013)
spray_dates=list(spray['Date'].unique())
spray_dates

['2011-08-29',
 '2011-09-07',
 '2013-07-17',
 '2013-07-25',
 '2013-08-08',
 '2013-08-15',
 '2013-08-16',
 '2013-08-22',
 '2013-08-29',
 '2013-09-05']

In [36]:
# Let's create a running count of days within the year:
raw['DayRunner']=pd.to_datetime(raw['Date']).dt.dayofyear
print(raw['DayRunner'].describe())
raw[['DayRunner', 'MonthDay', 'Month', 'Day']].head()

count    14835.00000
mean       227.46269
std         17.78001
min        198.00000
25%        206.00000
50%        227.00000
75%        241.00000
max        250.00000
Name: DayRunner, dtype: float64


,DayRunner,MonthDay,Month,Day
0,241,0829,08,29
1,241,0829,08,29
2,241,0829,08,29
3,241,0829,08,29
4,241,0829,08,29


In [37]:
spray_days=list(raw['DayRunner'].unique().astype(int))
spray_days

[241, 250, 198, 206, 220, 227, 228, 234, 248]

In [38]:
spray_downs=[x + 15 for x in spray_days]
spray_downs

[256, 265, 213, 221, 235, 242, 243, 249, 263]

In [39]:
spray_windows=zip(spray_days, spray_downs)
list(spray_windows)

[(241, 256),
 (250, 265),
 (198, 213),
 (206, 221),
 (220, 235),
 (227, 242),
 (228, 243),
 (234, 249),
 (248, 263)]

In [34]:
# Any day-runner that falls within these window should be considered to have the spray effect.
# Note that the first two are 2011, the rest are 2013.
# Of course, they aren't all in the same location.

In [35]:
# This info has been incorporated into the trap_readings dataset.